# Workshop: Vectorized Backtesting & Alpha Generation
**Goal:** Build a Mean-Reversion strategy from scratch. We will use Pandas for signal generation and pure NumPy for high-performance backtesting.

In [ ]:
#main imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#for data
import yfinance as yf

#plotting config
plt.style.use('dark_background')
plt.rcParams['figure.figsize'] = (14, 7)

## 1. Getting the Data
Lets pull some data that experiences trends and fluctuations, so that we can see the mean and its deviations.

In [ ]:
#fetch data
ticker = 'SPY'
df = yf.download(ticker, start='2021-01-01', end='2026-01-01')

#to fix the multi index columns
if isinstance(df.columns, pd.MultiIndex):
    df.columns = df.columns.get_level_values(0)

#get close prices
df = df[['Close']].copy()

#inspect data
display(df.head())
df.info()

## 2. Making Signals with Pandas (Bollinger Bands / Z-Score)
We are going to build a Z-Score indicator. This tells us how many standard deviations the current price is away from its rolling mean.
* **Assumption:** The price will eventually revert to its historical mean.

In [ ]:
window = 20 #arbitrary rolling window

#we use .rolling rather than a loop
df['SMA'] = df['Close'].rolling(window=window).mean()
df['Rolling_Std'] = df['Close'].rolling(window=window).std()

#calculate z score (close - mean) / std
df['Z_Score'] = (df['Close'] - df['SMA']) / df['Rolling_Std']

#drop garbage values
df.dropna(inplace=True)

df[['Close', 'SMA']].plot(title=f'{ticker} Price vs {window}-Day SMA')
plt.show()

## 3. Making Trades (Avoiding Look-Ahead Bias)
Now we define our logic:
* **Go Long (+1):** If Z-Score < -2 (Oversold)
* **Go Short (-1):** If Z-Score > 2 (Overbought)
* **Flat (0):** Otherwise

**CRITICAL RULE:** If we calculate our signal using today's `Close` price, we cannot execute our trade *at* today's `Close` price. That creates data leakage (peeking into the future). We must execute tomorrow. We simulate this by shifting our position vector forward by 1.

In [ ]:
#np.where(condition, value_if_true, value_if_false) is good for making masks
#we nest them to handle Long, Short, and Flat states
#we have (1, -1, 0) for (Long, Short, Flat)
df['Signal'] = np.where(df['Z_Score'] < -2, 1, 
               np.where(df['Z_Score'] > 2, -1, 0))

#shift the signal to create the trade
df['Position'] = df['Signal'].shift(1)

#normally we forward fill to hold until next signal
df['Position'] = df['Signal'].shift(1).ffill()

#drop the first row since it's NaN
df.dropna(inplace=True) 

print(df['Position'].value_counts())

## 4. Vectorized Backtesting with NumPy
Pandas is great for aligning data, but NumPy arrays are stored in contiguous blocks of memory, making mathematical operations lightning fast. We will extract our columns to raw NumPy arrays (`.values`) to calculate PnL.

In [ ]:
#calculate the market returns (we do this wth log returns)
market_returns = np.log(df['Close'] / df['Close'].shift(1)).fillna(0).values

#get our positions from earilier
positions = df['Position'].values

#we can very quickly backtest by just multiplying our positions by the market returns
#consider is as the movement in the market weighted by exposure to the market
strategy_returns = positions * market_returns

#then we can just sum up all those little movements and exponentiate to get back the value
cum_market_returns = np.exp(np.cumsum(market_returns)) - 1
cum_strategy_returns = np.exp(np.cumsum(strategy_returns)) - 1

#lets calculate the sharpe ratio (assuming 252 trading days)
#sharpe = (mean return / volatility) * sqrt(252)
mean_strat_ret = np.mean(strategy_returns)
std_strat_ret = np.std(strategy_returns)
sharpe_ratio = (mean_strat_ret / std_strat_ret) * np.sqrt(252)

print(f"Strategy Annualized Sharpe Ratio: {sharpe_ratio:.2f}")

## 5. Visualizing the Equity Curve
Lets plot the equity value to see how we did.

In [ ]:
plt.figure(figsize=(14, 7))

#we need to pass in the index to accurately plot the dates
plt.plot(df.index, cum_market_returns, label='Buy & Hold SPY Benchmark', color='gray', alpha=0.7)
plt.plot(df.index, cum_strategy_returns, label='Mean Reversion Strategy', color='cyan', linewidth=2)

plt.title('Strategy vs Benchmark Equity Curve')
plt.xlabel('Date')
plt.ylabel('Cumulative Return')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()